In [ ]:
import numpy as np
import os       # to work with files
import cv2      # to read and resize images
import matplotlib.pyplot as plt
import pandas as pd
import pickle   # for storage option

#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()
#(or)
#tf.compat.v1.disable_v2_behavior()
#from tensorflow import keras
#from tqdm import tqdm_notebook   # help visualize training process
from tqdm.notebook import tqdm_notebook
from scipy.spatial.distance import hamming, cosine   # to compare vectors

# keep all the visualizations into the notebook file
%matplotlib inline

### Data preprocessing
###### Load image + Preprocess dataset

##### 1 - Load image

In [2]:
def image_loader(image_path, image_size):
    '''
    Load an image from the disk.
    
    image_path:string -> path to the image
    image_size:tuple -> size of an output image, example : (32, 32)
    '''
    
    # to read an image from the disk
    image = cv2.imread(image_path)
    
    # cv2 library loads images in BGR format, need to convert it to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # resizing the image
    # here we use CUBIC interpolation (way to fill a missing data) to resize the image
    image = cv2.resize(image, image_size, cv2.INTER_CUBIC)
    
    return image    

##### 2 - Preprocess dataset

In [3]:
def dataset_preprocessing(dataset_path, labels_file_path, image_size, image_paths_pickle):
    '''
    Load images and labels from the folder containing the dataset.
    
    dataset_path:string -> path to the train/test dataset folder
    labels_file_path:string -> path to the .txt file containing classes names
    image_size:tuple -> single image size
    image_paths_pickle:string -> name of a pickle file where all image paths will be saved
    '''
    
    # get all classes in the dataset
    with open(labels_file_path, "r") as f:
        classes = f.read().split("\n")[:-1]  # list of all 10 classes. each line is a class name.
        # [:-1] to remove the last element which is a ""
    #print(classes)
    
    images = []  # hold all the images
    labels = []  # hold all respective classes for each image
    image_paths = []   # hold path for each image, list is stored in the pickle file
    
    # go through all the images in the dataset folder
    for image_name in os.listdir(dataset_path):
        try:
            image_path = os.path.join(dataset_path, image_name)   # example : cifar/train/0_frog.png
            images.append(image_loader(image_path, image_size))
            image_paths.append(image_path)  # append the images' paths to the paths list
            
            # go through all the classes and check if the class name appears in the image name
            for index in range(len(classes)):
                if classes[index] in image_name:   # ex° : if 'frog' is in '0_frog.png'
                    labels.append(index)   # if there's a match, we append the index in the list of labels
                    # labels are encoded in this list
                    
        except:
            pass
    
    # save all the paths in the pickle
    with open(image_paths_pickle + ".pickle", "wb") as f:
        pickle.dump(image_paths, f)
    
    assert len(images) == len(labels)   # make sure same amount of elements in 'images' and 'labels' lists
    # if this statement is false, the rest of the code won't be executed.
    
    return np.array(images), np.array(labels)

    

In [33]:
np.array(labels)

array([6, 1, 6, ..., 5, 1, 3])

In [4]:
images, labels = dataset_preprocessing("cifar/train/", \
                      "cifar/labels.txt", \
                      (32, 32), \
                      "cifar/training_images_pickle"                      
                     )

#(takes some time)

In [5]:
images.shape
# (50000, 32, 32, 3)
# there are 50000 images of size 32x32 in RGB format

(50000, 32, 32, 3)

### Vectors distances functions

In [6]:
# Cosine distance
# to compare images -> see similarities between vectors by measuring the cosine distance
'''
Calculates cosine distances between query image (vectors) and all training set images (vectors).

training_set_vectors:numpy Matrix -> vectors for all images in the training set
query_vector:numpy vector -> query image (new image) vector
top_n=50:integer -> nb of closest images to return
'''

def cosine_distance(training_set_vectors, query_vector, top_n=50):
    
    distances = []   # all the calculated distances stored in the list
    
    for i in range(len(training_set_vectors)):   # 50.000 images for Cifar-10 ds
        distances.append(cosine(training_set_vectors[i], query_vector[0]))    # cosine(u, v)
        
    return np.argsort(distances)[:top_n]   #sorted indices
    # takes an array and sort it, and returns sorted indices instead of a sorted array


In [7]:
# Hamming distance
# works in binary vectors only, unlike cosine distance
'''
Calculates hamming distances between query image (vectors) and all training set images (vectors).

training_set_vectors:numpy Matrix -> vectors for all images in the training set
query_vector:numpy vector -> query image (new image) vector
top_n=50:integer -> nb of closest images to return
'''

def hamming_distance(training_set_vectors, query_vector, top_n=50):
    
    distances = [] 
    
    for i in range(len(training_set_vectors)):   # 50.000 images for Cifar-10 ds
        distances.append(hamming(training_set_vectors[i], query_vector))  
        
    return np.argsort(distances)[:top_n]  


### Accuracy function

In [8]:
# Sparse accuracy to check how accurate our model is (how often it predicts the correct class)
''' 
Calculates accuracy of a model based on softmax outputs.

true_label:numpy array -> real labels of each sample
predicted_labels:numpy matrix -> softmax probabilities
'''

def sparse_accuracy(true_labels, predicted_labels):
    
    assert len(true_labels) == len(predicted_labels)
    
    correct = 0
    for i in range(len(true_labels)):
        if np.argmax(predicted_labels[i]) == true_labels[i]:
        # takes an array and return index of highest proba and compare it to the index of the true label
        # if true, add one to the correct predictions
            correct += 1
        
    return correct / len(true_labels)

### Model inputs function

In [9]:
import tensorflow.compat.v1 as tf   # by default in TF2, tf.executing_eagerly() -> True
print(tf.__version__)

2.4.1


In [10]:
tf.disable_v2_behavior()   # that makes tf.executing_eagerly() -> False
#tf.enable_eager_execution()   # that re-makes tf.executing_eagerly() -> True again
tf.executing_eagerly()

Instructions for updating:
non-resource variables are not supported in the long term


False

In [11]:
def model_inputs(image_size):
    '''
    Define CNN inputs (placeholders).
    
    image_size:tuple -> (height, width) of an image  # here (32, 32)
    '''
    
    # for TensorFlow 1 :
    # placeholder that receives the images
    inputs = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None, image_size[0], image_size[1], 3], name='images')
    # 'None' is the Batch size and it allows us to import any nb of images
    
    targets = tf.compat.v1.placeholder(dtype=tf.int32, shape=[None], name='targets')
    
    dropout_rate = tf.compat.v1.placeholder(dtype=tf.float32, name='dropout_rate')

    return inputs, targets, dropout_rate

### Convolutional block function

In [12]:
# Convolutional block function

def conv_block(inputs,
               number_of_filters,
               kernel_size,
               strides=(1, 1),
               padding='SAME',
               activation=tf.nn.relu,
               max_pool=True,
               batch_norm=True):
        
    '''
    Defines convolutional block layer.
    
    inputs -> data from previous layer
    number_of_filters:integer -> number of conv filters
    kernel_size:tuple -> size of conv layer kernel
    padding:string -> type of padding techniques: SAME or VALID
    activation:tf.object -> activ function used on the layer
    max_pool:boolean -> if True the conv block will use batch norm
    '''
        
    conv_features = layer = tf.layers.conv2d(inputs=inputs, 
                                             filters=number_of_filters, 
                                             kernel_size=kernel_size, 
                                             strides=strides, 
                                             padding=padding,
                                             activation=activation)
    
    if max_pool:
        layer = tf.layers.max_pooling2d(layer,
                                        pool_size=(2, 2),
                                        strides=(2, 2),
                                        padding='SAME')    
        
    if batch_norm:
        layer = tf.layers.batch_normalization(layer)
        
        
    return layer, conv_features


### Dense block function (fully connected layers)

In [13]:
#@tf.function
def dense_block(inputs, 
                units, 
                activation=tf.nn.relu, 
                dropout_rate=None, 
                batch_norm=True):

    '''
    Defines dense block layer.
    
    inputs -> data from previous layer
    units:integer -> nb of neurons/units for a dense layer
    activation:tf.object -> activ function used on the layer
    dropout_rate -> dropout rate used in this dense block
    batch_norm:boolean -> if True the conv block will use batch normalization
    '''
        
    dense_features = layer = tf.layers.dense(inputs=inputs,
                                                   units=units,
                                                   activation=activation)
    

        
    if dropout_rate is not None:
        layer = tf.layers.dropout(layer,
                                  rate=dropout_rate)
                                         
    
    if batch_norm:
        layer = tf.layers.batch_normalization(layer)

    return layer, dense_features
    

### Optimization and loss functions

In [14]:
def opt_loss(logits,
            targets,
            learning_rate):
    
    '''
    Defines model's optimizer and loss functions.
    
    logits -> pre-activated outputs from the last layer of the model
    targets -> true labels for each input sample
    learning_rate
    '''

    # sparse -> we don't have to convert our targets to one hot encoding versions
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=targets, logits=logits))
    
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    
    return loss, optimizer

### Model architecture

In [15]:
class ImageSearchModel(object):
    
    def __init__(self,
                learning_rate,
                image_size,
                number_of_classes=10):
        '''
        Defines CNN model.
        
        learning_rate
        image_size:tuple -> (height, width) of an image
        number_of_classes:integer -> nb of classes in a ds
        '''

        # reset the tensorflow graph each time the model is defined to avoid nested models
        #tf.reset_default_graph()
        #tf.compat.v1.reset_default_graph()
        
        # define model inputs
        self.inputs, self.targets, self.dropout_rate = model_inputs(image_size)
        # 'model_inputs' function is defined above, it returns 3 elements
        
        # no function for image normalization so they are between 0 and 255
        # to overcome this : batch normalization layer before the conv layer :
       
        normalized_images = tf.layers.batch_normalization(self.inputs)  #deprecated
        
        # conv block 1
        conv_block_1, self.conv_1_features = conv_block(inputs=normalized_images,
                                                        number_of_filters=64,
                                                        kernel_size=(3, 3),
                                                        strides=(1, 1),
                                                        padding='SAME',
                                                        activation=tf.nn.relu,
                                                        max_pool=True,
                                                        batch_norm=True)
                                          
        # conv block 2
        conv_block_2, self.conv_2_features = conv_block(inputs=conv_block_1,
                                                       number_of_filters=128,     
                                                       kernel_size=(3, 3),
                                                       strides=(1, 1),
                                                       padding='SAME',
                                                       activation=tf.nn.relu,
                                                       max_pool=True,
                                                       batch_norm=True)
        
        # conv block 3
        conv_block_3, self.conv_3_features = conv_block(inputs=conv_block_2,
                                                       number_of_filters=256,     
                                                       kernel_size=(5, 5),
                                                       strides=(1, 1),
                                                       padding='SAME',
                                                       activation=tf.nn.relu,
                                                       max_pool=True,
                                                       batch_norm=True)
        
        # conv block 4
        conv_block_4, self.conv_4_features = conv_block(inputs=conv_block_3,
                                                       number_of_filters=512,     
                                                       kernel_size=(5, 5),
                                                       strides=(1, 1),
                                                       padding='SAME',
                                                       activation=tf.nn.relu,
                                                       max_pool=True,
                                                       batch_norm=True)
        
        # before apply dense block, need to flatten conv features (reshape it to a single vector)
        flat_layer = tf.layers.flatten(conv_block_4)
        
        # dense block 1
        dense_block_1, dense_1_features = dense_block(flat_layer,
                                                     units=128,
                                                     activation=tf.nn.relu,
                                                     dropout_rate=self.dropout_rate,
                                                     batch_norm=True)
        
        # dense block 2
        dense_block_2, dense_2_features = dense_block(dense_block_1,
                                                     units=256,
                                                     activation=tf.nn.relu,
                                                     dropout_rate=self.dropout_rate,
                                                     batch_norm=True)
        
        # dense block 3
        dense_block_3, dense_3_features = dense_block(dense_block_2,
                                                     units=512,
                                                     activation=tf.nn.relu,
                                                     dropout_rate=self.dropout_rate,
                                                     batch_norm=True)
        
        # dense block 4
        dense_block_4, dense_4_features = dense_block(dense_block_3,
                                                     units=1024,
                                                     activation=tf.nn.relu,
                                                     dropout_rate=self.dropout_rate,
                                                     batch_norm=True)
        
        logits = tf.layers.dense(inputs=dense_block_4,
                                units=number_of_classes,
                                activation=None)
        
        self.predictions = tf.nn.softmax(logits)
        
        self.loss, self.opt = opt_loss(logits=logits,
                                      targets=self.targets,
                                      learning_rate=learning_rate)


In [16]:
model = ImageSearchModel(0.001, (32, 32), 10)

C:\Users\camib\anaconda3\lib\site-packages\tensorflow\python\keras\legacy_tf_layers\normalization.py:307: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  warnings.warn(
C:\Users\camib\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
C:\Users\camib\anaconda3\lib\site-packages\tensorflow\python\keras\legacy_tf_layers\convolutional.py:414: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  warnings.warn('`tf.layers.conv2d` is deprecated and '
C

### Training function

In [17]:
def train(model,
         epochs,
         drop_rate,
         batch_size,
         data,
         save_dir,
         saver_delta=0.15):
    
    '''
    Train the model.
    
    model -> CNN model
    epochs:int -> nb of epochs
    drop_rate:float -> dropout rate
    batch_size:int -> nb of samples to put through the model at once
    data:tuple -> train/test data , ex° X_train, y_train, X_test, y_test
    save_dir:string -> path to a folder where model checkpoints will be saved
    saver_delta:float -> trick used to prevet overfitted model to be saved
    '''
    
    X_train, y_train, X_test, y_test
    
    # start tensorflow session
    session = tf.Session()
    session.run(tf.global_variables_initializer())   # initialize all variables of the model
    
    # define saver
    saver = tf.train.Saver()   # if the trained model gives better accuracy than previously we save it
    
    best_test_accuracy = 0    # initialized to zero. It help us to decide whether we should save the model or not
    
    # Train the model
    for epoch in range(epochs):
        train_accuracy = []
        train_loss = []
        
        # split the set into batches
        for ii in tqdm_notebook(range(len(X_train) // batch_size)):
            start_id = ii * batch_size
            end_id = start_id + batch_size
            
            X_batch = X_train[start_id:end_id]
            y_batch = y_train[start_id:end_id]
            # so that now we have a batch of data prepared to feed the model
            
            feed_dict = {model.inputs:X_batch,
                        model.targets:y_batch,
                        model.dropout_rate:drop_rate}
            
            # we try to optimize our model using the batch of data
            # we return the optimizer, loss and predictions for the batch of data
            # 'session.run' to run the optimization step. (Second argument is to provide the data)
            _, training_loss, training_predictions = session.run([model.opt, model.loss, model.predictions], feed_dict=feed_dict)
            
            # we append the results to the accuracy list and loss list
            train_accuracy.append(sparse_accuracy(y_batch, training_predictions))
            train_loss.append(training_loss)
            
        print("Epoch: {}/{}".format(epoch, epochs),
             " | Training accuracy: {}".format(np.mean(train_accuracy)),
             " | Training loss: {}".format(np.mean(train_loss)) )
        
        
        # Test the model
        test_accuracy = []

        for ii in tqdm_notebook(range(len(X_test) // batch_size)):
            start_id = ii * batch_size
            end_id = start_id + batch_size

            X_batch = X_test[start_id:end_id]
            y_batch = y_test[start_id:end_id]

            # for testing process we only need the predictions, not the targets anymore
            # we set the dropout rate to zero. We thus get all weights without dropping any
            feed_dict = {model.inputs:X_batch,
                                model.dropout_rate:0.0}

            test_predictions = session.run(model.predictions, feed_dict=feed_dict)
            test_accuracy.append(sparse_accuracy(y_batch, test_predictions))  # test accuracy between true targets and predictions

        print("Test accuracy: {}".format(np.mean(test_accuracy)))

                
        # Saving the model
        # check if the training accuracy is higher than the test accuracy
        # it avoids saving underfitted model
        # and overfitted model (thanks to a delta)

        if np.mean(train_accuracy) > np.mean(test_accuracy):
            if np.abs(np.mean(train_accuracy) - np.mean(test_accuracy)) <= saver_delta:
                if np.mean(test_accuracy) >= best_test_accuracy:
                    best_test_accuracy = np.mean(test_accuracy)
                    saver.save(session, "{}/model_epoch_{}.ckpt".format(save_dir, epoch))  # save the model
                    # 'ckpt' is checkpoint

    session.close()

### Training the model
#### Setting hyperparameters and training the model

In [18]:
epochs = 20
batch_size = 128
learning_rate = 0.001
dropout_rate = 0.6
image_size = (32, 32)  # for Cifar-10 ds

In [19]:
X_train, y_train = dataset_preprocessing("cifar/train/", "cifar/labels.txt", image_size=image_size, image_paths_pickle="train_images_pickle")

In [20]:
X_train.shape

(50000, 32, 32, 3)

In [21]:
X_test, y_test = dataset_preprocessing("cifar/test/", "cifar/labels.txt", image_size=image_size, image_paths_pickle="test_images_pickle")

In [22]:
X_test.shape

(10000, 32, 32, 3)

In [23]:
# Define the model
model = ImageSearchModel(learning_rate=learning_rate, image_size=image_size)

In [24]:
data = (X_train, y_train, X_test, y_test)   # tuple, consists in all training and test subsets

In [25]:
train(model, epochs, dropout_rate, batch_size, data, "saver")
# 'saver' : folder created manually in which all the chekpoints are saved

  0%|          | 0/390 [00:00<?, ?it/s]

Epoch: 0/20  | Training accuracy: 0.37029246794871795  | Training loss: 1.8305789232254028


  0%|          | 0/78 [00:00<?, ?it/s]

Test accuracy: 0.5176282051282052


  0%|          | 0/390 [00:00<?, ?it/s]

...

Test accuracy: 0.6802884615384616


  0%|          | 0/390 [00:00<?, ?it/s]

Epoch: 19/20  | Training accuracy: 0.9559294871794872  | Training loss: 0.1381281018257141


  0%|          | 0/78 [00:00<?, ?it/s]

Test accuracy: 0.6854967948717948


In [26]:
def create_training_set_vectors(model,
                               X_train,
                               y_train,
                               batch_size,
                               checkpoint_path,
                               image_size,
                               distance='hamming'):
    
        session = tf.Session()
        session.run(tf.global_variables_initializer())
        
        saver = tf.train.Saver()
        saver.restore(session, checkpoint_path)
        
        dense_2_features = []
        dense_4_features = []
        
        for ii in tqdm_notebook(range(len(X_train) // batch_size)):
            start_id = ii * batch_size
            end_id = start_id + batch_size
            
            X_batch = X_train[start_id:end_id]
            
            feed_dict = {model.inputs:X_batch, model.dropout_rate:0.0}
            
            dense_2, dense_4 = session.run([model.dense_2_features, model.dense_4_features], feed_dict=feed_dict)
            
            dense_2_features.append(dense_2)
            dense_4_features.append(dense_4)
            
        dense_2_features = np.vstack(dense_2_features)
        dense_4_features = np.vstack(dense_4_features)
        
        if distance == 'hamming':
            dense_2_features = np.where(dense_2_features < 0.5, 0, 1)
            dense_4_features = np.where(dense_4_features < 0.5, 0, 1)
            
            training_vectors = np.hstack((dense_2_features, dense_4_features))
            
            with open("hamming_train_vectors.pickle", "wb") as f:
                pickle.dump(training_vectors, f)
                
        elif distance == "cosine":
            training_vectors = np.hstack((dense_2_features, dense_4_features))
            
            with open("hamming_train_vectors.pickle", "wb") as f:
                pickle.dump(training_vectors, f)
                
            

In [27]:
model = ImageSearchModel(learning_rate, image_size)

In [29]:
create_training_set_vectors(model, X_train, y_train, batch_size, "saver/model_epoch_6.ckpt", image_size)

INFO:tensorflow:Restoring parameters from saver/model_epoch_6.ckpt


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key batch_normalization_18/beta not found in checkpoint
	 [[node save_1/RestoreV2 (defined at <ipython-input-26-b109b43aa80b>:12) ]]

Original stack trace for 'save_1/RestoreV2':
  File "C:\Users\camib\anaconda3\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\camib\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\camib\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\camib\anaconda3\lib\site-packages\traitlets\config\application.py", line 845, in launch_instance
    app.start()
  File "C:\Users\camib\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 612, in s

In [ ]:
def simple_inferences(model,
                     session,
                     train_set_vectors,
                     uploaded_image_path,
                     image_size,
                     distance='hamming'):
    
    image = image_loader(uploaded_image_path, image_size)
    
    feed_dict = {model.inputs:[image], model.dropout_rate:0.0}
    
    dense_2_features, dense_4_features = session.run([model.dense_2_features, model.dense_4_features], feed_dict=feed_dict)
    
    closest_ids = None
    if distance == 'hamming':
        dense_2_features = np.where(dense_2_features < 0.5, 0, 1)
        dense_4_features = np.where(dense_4_features < 0.5, 0, 1)
            
        uploaded_image_vector = np.hstack((dense_2_features, dense_4_features))
            
        closest_ids = hamming_distance(train_set_vectors, uploaded_image_vector)
        
    elif distance == "cosine":
        uploaded_image_vector = np.hstack((dense_2_features, dense_4_features))
        
        closest_ids = cosine_distance(train_set_vectors, uploaded_image_vector)
        
    return closest_ids

In [ ]:
model = ImageSearchModel(learning_rate=learning_rate, image_size=image_size)

session = tf.Session()
session.run(tf.global_variables_initializer())

In [ ]:
saver = tf.train.Saver()
saver.restore(session, "saver/model_epoch_5.ckpt")

In [ ]:
with open('train_images_pickle.pickle', 'rb') as f:
    train_image_paths = pickle.load(f)

In [ ]:
train_image_paths[:10]

In [ ]:
with open('hamming_train_vectors.pickle', 'rb') as f:
    train_set_vectors = pickle.load(f)

In [ ]:
test_image = 'dataset/test/1052_airplane.png'

In [ ]:
result_ids = simple_inference(model, session, train_set_vectors, test_image, image_size, distance='hamming')

In [ ]:
plt.imshow(image_loader(test_image, image_size))

In [ ]:
w = 10
h = 10
fig = plt.figure(figsize=(16, 16))
columns = 10
rows = 5
for i in range(1, columns*rows+1):
    try:
        image = image_loader(train_image_paths[result_ids[i]], image_size)
        fig.add_subplot(rows, columns, i)
        plt.imshow(image)
    except:
        pass
plt.show()